In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping
# from sklearn.metrics import mean_squared_error, mean_absolute_error

# from tensorflow.keras.models import Sequential
# from keras.layers import LSTM, Dropout, Dense, Input, Concatenate
# from tensorflow.keras.layers import Input
# from tensorflow.keras.models import Model
# from keras_tuner import HyperModel, RandomSearch
# from sklearn.metrics import r2_score
# from sklearn.model_selection import TimeSeriesSplit


In [3]:

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ta.trend import EMAIndicator, ADXIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands


import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

In [12]:
def collect_data():

    # Data Collection from yfinance
    # EUR/USD Data
    eurusd_daily = yf.download('EURUSD=X', period='10y', interval='1d')
    # VIX (Volatility Index)
    vix_daily = yf.download('^VIX', period='10y', interval='1d')
    # USDX (US Dollar Index)
    usdx_daily = yf.download('DX-Y.NYB', period='10y', interval='1d')
    # SP500 data
    sp_daily = yf.download('^GSPC', period='10y', interval='1d')

    # Macroeconomic Proxies from yfinance
    # Non-Farm Payroll proxy (XLI - Industrials ETF)
    xli = yf.download('XLI', period='10y', interval='1d')
    # CPI proxy (TIP - iShares TIPS Bond ETF)
    tip = yf.download('TIP', period='10y', interval='1d')
    # Treasury Yield proxy (TLT - iShares 20+ Year Treasury Bond ETF)
    tlt = yf.download('TLT', period='10y', interval='1d')
    # Interest Rate proxy (SHY - iShares 1-3 Year Treasury Bond ETF)
    shy = yf.download('SHY', period='10y', interval='1d')

    # Merge macroeconomic data into one DataFrame
    macro_data = pd.DataFrame({
        'XLI_Close': xli['Close'],
        'TIP_Close': tip['Close'],
        'TLT_Close': tlt['Close'],
        'SHY_Close': shy['Close']
    })

    macro_data.index = pd.to_datetime(macro_data.index)
    eurusd_daily = eurusd_daily.merge(macro_data, how='inner', left_index=True, right_index=True)

    return eurusd_daily, macro_data


# Technical Indicator Calculation
def add_indicators(data):
    # Add EMA, ADX, MACD, RSI, and Bollinger Bands
    ema1 = EMAIndicator(close=data['Close'], window=7)
    ema2 = EMAIndicator(close=data['Close'], window=21)
    data['EMA7'] = ema1.ema_indicator()
    data['EMA21'] = ema2.ema_indicator()

    macd = MACD(close=data['Close'])
    data['MACD'] = macd.macd_signal()

    rsi = RSIIndicator(close=data['Close'], window=14)
    data['RSI'] = rsi.rsi()

    bb = BollingerBands(close=data['Close'], window=20)
    data['bb_mavg'] = bb.bollinger_mavg()
    data['bb_upper'] = bb.bollinger_hband()
    data['bb_lower'] = bb.bollinger_lband()

    return data.dropna()



# Plot signals with Bollinger Bands and RSI
def plot_subplots(data, signal_col):
    """
    Create subplots for EUR/USD price data, including Bollinger Bands and RSI signals.

    Parameters:
        data (DataFrame): DataFrame containing price data and signals.
        signal_col (str): Column name for the signal ('Buy', 'Sell', 'Hold').
    """
    # Filter out signals that are 'Hold'
    df_signals = data[data[signal_col] != 'Hold']

    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        row_heights=[800, 200],
        shared_xaxes=True,
        vertical_spacing=0.02,
        specs=[[{"secondary_y": True}], [{"secondary_y": False}]]
    )

    # Add Candlestick trace
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            name='Candlestick'
        ),
        secondary_y=True,
        row=1, col=1
    )

    # Add Bollinger Bands traces
    fig.add_trace(
        go.Scatter(x=data.index, y=data["bb_lower"], name='Lower Bollinger Band',
                   line=dict(color='royalblue', width=1)),
        secondary_y=True, row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=data.index, y=data['bb_upper'], name='Upper Bollinger Band',
                   line=dict(color='royalblue', width=1)),
        secondary_y=True, row=1, col=1
    )

    # Add Buy/Sell annotations
    for i, dtt in df_signals.iterrows():
        if dtt[signal_col] == 'Buy':
            fig.add_annotation(
                x=i,
                y=dtt['Low'] - 0.05,
                xref='x',
                yref='y',
                text="▲",
                showarrow=False,
                font=dict(size=20, color='Green'),
                secondary_y=True,
                row=1, col=1
            )
        elif dtt[signal_col] == 'Sell':
            fig.add_annotation(
                x=i,
                y=dtt['High'] + 0.05,
                xref='x',
                yref='y',
                text="▼",
                showarrow=False,
                font=dict(size=20, color='Red'),
                secondary_y=True,
                row=1, col=1
            )

    # Add RSI trace
    fig.add_trace(
        go.Scatter(x=data.index, y=data.RSI,
                   line=dict(color='red', width=1),
                   mode='lines',
                   name='RSI'),
        row=2, col=1
    )

    # Add horizontal lines for RSI thresholds
    fig.add_hline(y=30.0, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=70.0, line_dash="dash", line_color="green", row=2, col=1)

    # Update layout settings
    fig.update_layout(
        showlegend=False,
        margin=dict(l=50, r=1, t=50, b=10),
        title_text=f"{signal_col} - EUR/USD with Bollinger Bands and RSI",
        xaxis_title="Date",
        yaxis_title="Price",
        height=500
    )

    # Customize axes appearance
    fig.update_xaxes(showgrid=False, showspikes=True, rangebreaks=[
            dict(bounds=["sat", "mon"])  # Hide weekends
        ])

    fig.update_yaxes(showgrid=False)

    # Show the figure
    fig.show()

In [5]:
eurusd_daily, macro_data = collect_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
eurusd_daily

,Open,High,Low,Close,Adj Close,Volume,XLI_Close,TIP_Close,TLT_Close,SHY_Close
Date,,,,,,,,,,
2014-10-27,1.267684,1.272200,1.266709,1.267668,1.267668,0,53.689999,113.230003,119.940002,84.820000
2014-10-28,1.270245,1.276300,1.268698,1.270148,1.270148,0,54.639999,113.180000,119.190002,84.800003
2014-10-29,1.273804,1.276979,1.272508,1.273837,1.273837,0,54.450001,113.029999,119.459999,84.690002
2014-10-30,1.263232,1.263408,1.255870,1.263280,1.263280,0,54.669998,112.989998,119.559998,84.720001
2014-10-31,1.261002,1.261002,1.249126,1.261336,1.261336,0,55.209999,113.080002,119.250000,84.699997
...,...,...,...,...,...,...,...,...,...,...
2024-10-21,1.086862,1.086957,1.082111,1.086862,1.086862,0,138.779999,108.570000,92.230003,82.470001
2024-10-22,1.081783,1.083776,1.080194,1.081783,1.081783,0,137.100006,108.720001,92.320000,82.459999
2024-10-23,1.079727,1.080847,1.076264,1.079727,1.079727,0,136.699997,108.449997,92.070000,82.410004


In [7]:
macro_data

,XLI_Close,TIP_Close,TLT_Close,SHY_Close
Date,,,,
2014-10-27,53.689999,113.230003,119.940002,84.820000
2014-10-28,54.639999,113.180000,119.190002,84.800003
2014-10-29,54.450001,113.029999,119.459999,84.690002
2014-10-30,54.669998,112.989998,119.559998,84.720001
2014-10-31,55.209999,113.080002,119.250000,84.699997
...,...,...,...,...
2024-10-21,138.779999,108.570000,92.230003,82.470001
2024-10-22,137.100006,108.720001,92.320000,82.459999
2024-10-23,136.699997,108.449997,92.070000,82.410004


In [8]:
eurusd_daily = add_indicators(eurusd_daily)
print("Technical Indicators Added:")

# Access the correct column names for Bollinger Bands: 'bb_upper' and 'bb_lower'
eurusd_daily[['Close', 'EMA7', 'EMA21', 'RSI', 'bb_upper', 'bb_lower']]


Technical Indicators Added:


,Close,EMA7,EMA21,RSI,bb_upper,bb_lower
Date,,,,,,
2014-12-12,1.239495,1.239600,1.243245,46.122280,1.258056,1.228810
2014-12-15,1.246354,1.241288,1.243528,50.770896,1.257933,1.228809
2014-12-16,1.244354,1.242055,1.243603,49.431196,1.257043,1.228949
2014-12-17,1.251204,1.244342,1.244294,53.915370,1.257792,1.228831
2014-12-18,1.233898,1.241731,1.243349,43.436303,1.256605,1.228074
...,...,...,...,...,...,...
2024-10-21,1.086862,1.089118,1.097613,33.647974,1.125095,1.078249
2024-10-22,1.081783,1.087284,1.096174,29.639945,1.123924,1.075989
2024-10-23,1.079727,1.085395,1.094679,28.176983,1.123309,1.073460


In [9]:
# Signal Generation
def generate_signals(data):
    # data['Buy_Signal'] = (data['RSI'] < 30) & (data['Close'] < data['bb_lower'])
    # data['Sell_Signal'] = (data['RSI'] > 70) & (data['Close'] > data['bb_upper'])

    data['Signal'] = np.where((data['RSI'] < 30) & (data['Close'] < data['bb_lower']), 'Buy', 'Hold')
    data['Signal'] = np.where((data['RSI'] > 70) & (data['Close'] > data['bb_upper']), 'Sell', data['Signal'])
    return data

In [10]:
eurusd_daily = generate_signals(eurusd_daily)
eurusd_daily

,Open,High,Low,Close,Adj Close,Volume,XLI_Close,TIP_Close,TLT_Close,SHY_Close,EMA7,EMA21,MACD,RSI,bb_mavg,bb_upper,bb_lower,Signal
Date,,,,,,,,,,,,,,,,,,
2014-12-12,1.239127,1.247707,1.238669,1.239495,1.239495,0,54.820000,112.779999,126.300003,84.660004,1.239600,1.243245,-0.004821,46.122280,1.243433,1.258056,1.228810,Hold
2014-12-15,1.246479,1.248000,1.241710,1.246354,1.246354,0,54.639999,112.339996,126.040001,84.599998,1.241288,1.243528,-0.004565,50.770896,1.243371,1.257933,1.228809,Hold
2014-12-16,1.244354,1.256570,1.243626,1.244354,1.244354,0,54.689999,113.040001,127.599998,84.650002,1.242055,1.243603,-0.004261,49.431196,1.242996,1.257043,1.228949,Hold
2014-12-17,1.251377,1.251596,1.239010,1.251204,1.251204,0,55.200001,112.510002,126.449997,84.570000,1.244342,1.244294,-0.003823,53.915370,1.243311,1.257792,1.228831,Hold
2014-12-18,1.233806,1.235147,1.226670,1.233898,1.233898,0,56.590000,111.800003,124.199997,84.519997,1.241731,1.243349,-0.003592,43.436303,1.242340,1.256605,1.228074,Hold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-21,1.086862,1.086957,1.082111,1.086862,1.086862,0,138.779999,108.570000,92.230003,82.470001,1.089118,1.097613,-0.003696,33.647974,1.101672,1.125095,1.078249,Hold
2024-10-22,1.081783,1.083776,1.080194,1.081783,1.081783,0,137.100006,108.720001,92.320000,82.459999,1.087284,1.096174,-0.004230,29.639945,1.099957,1.123924,1.075989,Hold
2024-10-23,1.079727,1.080847,1.076264,1.079727,1.079727,0,136.699997,108.449997,92.070000,82.410004,1.085395,1.094679,-0.004748,28.176983,1.098384,1.123309,1.073460,Hold


In [12]:
eurusd_daily[eurusd_daily['Signal'] != 'Hold']

,Open,High,Low,Close,Adj Close,Volume,XLI_Close,TIP_Close,TLT_Close,SHY_Close,EMA7,EMA21,MACD,RSI,bb_mavg,bb_upper,bb_lower,Signal
Date,,,,,,,,,,,,,,,,,,
2015-01-05,1.195500,1.197590,1.188909,1.194643,1.194643,0,55.189999,112.820000,129.320007,84.540001,1.211807,1.225545,-0.007815,26.967448,1.228319,1.256824,1.199814,Buy
2015-01-06,1.193830,1.197000,1.188693,1.193902,1.193902,0,54.509998,112.830002,131.649994,84.580002,1.207331,1.222669,-0.008588,26.703641,1.226081,1.257830,1.194331,Buy
2015-01-07,1.187479,1.190000,1.180401,1.187536,1.187536,0,54.919998,112.919998,131.389999,84.620003,1.202382,1.219475,-0.009462,24.489385,1.223987,1.259839,1.188135,Buy
2015-01-23,1.134649,1.137398,1.112180,1.134675,1.134675,0,55.990002,114.320000,134.770004,84.849998,1.159321,1.184265,-0.017389,19.303513,1.186815,1.234087,1.139543,Buy
2015-01-26,1.114082,1.128399,1.114082,1.112545,1.112545,0,56.060001,114.129997,134.360001,84.800003,1.147627,1.177745,-0.018675,15.591751,1.181568,1.236699,1.126436,Buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-20,1.090703,1.094511,1.089800,1.090703,1.090703,0,105.930000,104.769997,90.589996,81.290001,1.081587,1.071736,0.003656,70.254395,1.068866,1.090327,1.047404,Sell
2023-11-21,1.094595,1.096732,1.092299,1.094595,1.094595,0,105.849998,104.550003,90.550003,81.339996,1.084839,1.073814,0.004406,72.367056,1.070246,1.094426,1.046066,Sell
2024-08-20,1.108647,1.111729,1.107248,1.108647,1.108647,0,127.389999,108.889999,98.669998,82.639999,1.100285,1.093336,0.003631,71.005079,1.091093,1.106843,1.075344,Sell


In [13]:
plot_subplots(eurusd_daily, 'Signal')